# Feast
Feast is a feature store helps you store and serve features consistently for offline training and online inference.  
With Feast, you can define, manage, validate, and serve features for production AI/ML, allowing both for collaboration around the features and to utilize features across AI products at scale.  

## Apply features to Feast
First we apply all features defined inside the `feature_repo`.  
This will store these feature definitions inside a database (referred to here as the Feast Registry). In our case, this is a simple postrgressql database.  
Inside the feature defintions we also point out where the data for these features is stored, so we know where and how to go and get it.

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
import feast
from datetime import datetime
import yaml, sys, os

In [ ]:
sys.path.append(os.path.abspath('feature_repo/'))
from features import music, song_properties
from feature_service import song_properties_fs

Whenever we use Feast, we need to point out the project, registry, online, and offline store so that we are using the corect data sources.  
A simple way to do that is to have a yaml file with the spec that we can referr to, but we will later move on to use Feast servers so we don't need this yaml everywhere we want to use Feast.

In [ ]:
with open('feature_repo/feature_store.yaml', 'r') as file:
    fs_config_yaml = yaml.safe_load(file)

fs_config = feast.repo_config.RepoConfig(**fs_config_yaml)
fs = feast.FeatureStore(config=fs_config)

When we apply features to Feast we point out what objects we want to apply. In our case it's:
- A Feature View - a collection of features where you freely can choose which ones you want to use, great for experimentation.
- A Feature Services - a logical group of features from one or more feature views. This is handy to use for production as it always point to the same group of features.

In [ ]:
fs.apply([song_properties_fs, music, song_properties])

## Materialize our features
Next step is to materialize the features, what this does is move the features from the offline store into the online store.  
Specifically, we move a subset of the features, the ones that's within a defined timeframe and we only store the latest features inside the feature store.

In [ ]:
fs.materialize(start_date=datetime(2023, 1, 1), end_date=datetime.now())

## Next step
Now that we have set up Feast, let's start using it!  
Go to the next notebook to see how we can fetch training features: [2-test_load_historitcal_features.ipynb](2-test_load_historitcal_features.ipynb)